# Cleaning OpenRoads topology

OpenRoads data are not always topologically correct, meaning that they introduce nodes of a degree 2. Topologically precise network has a single LineString between two junctions. This notebook fixes topology of OpenRoads data and saves resulting GeoDataFrame to PostGIS.

Note that `consolidate` is a [custom python file](https://github.com/urbangrammarai/data_processing/blob/master/vector_data/consolidate.py) containing definitions of required functions. Even though these were originally written for this project, in future they will be included in the existing Python ecosystem:

- `topology` will be included in [momepy 0.4](https://github.com/martinfleis/momepy) as `remove_false_nodes`

In [3]:
import os
import time

import geopandas as gpd

from sqlalchemy import create_engine

from consolidate import topology

user = os.environ.get('DB_USER')
pwd = os.environ.get('DB_PWD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT')

In [2]:
db_connection_url = f"postgres+psycopg2://{user}:{pwd}@{host}:{port}/built_env"
engine = create_engine(db_connection_url)
engine.begin()

We load the whole `openroads_200803` table from existing PostGIS database.

In [4]:
df = gpd.read_postgis('SELECT * FROM openroads_200803', engine, geom_col='geometry')
df.shape

(3719381, 21)

Using `topology` function stored in `consolidation.py` file, we clean the network by merging segments which introduces false junctions.

In [8]:
s = time.time()
fixed = topology(df)
print(time.time() - s)

4619.6336443424225


In [9]:
fixed.shape

(3569045, 21)

In [10]:
fixed.to_postgis("openroads_200803_topological", engine, if_exists='replace')